<a href="https://colab.research.google.com/github/moon-123/NLP-Okt/blob/main/%08%ED%95%9C%EA%B5%AD%EC%96%B4_NLP_%EA%B3%BC%EC%A0%9C(Okt%2C_%EB%AA%85%EC%82%AC%EB%A7%8C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 문화, 게임 콘텐츠 분야 용어 말뭉치

# 2. 데이터 불러오기

In [8]:
import json

with open('/content/drive/MyDrive/KDT/자연어 처리/data/용례_레저.json', 'r') as file:
    json_data = json.load(file)


In [9]:
json_data[0]

{'id': 3087876,
 'sentence': '김치볶음 해놓은거에 밥비벼먹고 카레 해먹고',
 'tokens': [{'start': 0,
   'length': 4,
   'sub': '김치볶음',
   'facet': '구체물',
   'term_id': 134217,
   'sense_no': 1}],
 'sense_no': 1,
 'source': {'uri': 'https://bbs.ruliweb.com/community/board/300143/read/58933395',
  'text': '일주일동안 탄수화물 위주로만 먹었어\r\n\r\n\r\n김치볶음 해놓은거에 밥비벼먹고 카레 해먹고\r\n\r\n\r\n점심에는 김밥에 쫄면에 돈까스 먹었는데\r\n\r\n\r\n그래도 단백질이 먹고싶다',
  'written_at': '2022-10-15T08:04:00'}}

### 2-1. 'sentence'
* 'sentence'는 source text에서 추출된 문장.

In [10]:
# source의 text를 정제/추출해도 되는데 과제는 전처리가 아니라 임베딩이므로 생략
# sentence를 사용
sentence_list = [data['sentence'] for data in json_data]
print(sentence_list[:5])
print(len(sentence_list))

['김치볶음 해놓은거에 밥비벼먹고 카레 해먹고', '전과하고 싶어지네 전적대에서 다니던 과로 전과할까 해도 이번학기 학점이 1~2점 나올거 같아서 재편입 해야하나 싶다...', '중요한건 저기 예시로 든건 노트북용도 아닌 컴터용 시퓨다', '의자를 좀 더 편한걸로 바꿔주면 딱이네 빈백 같은거', '수석이니하면서 장미빛 미래를 꿈꾸고 있을']
75384


### 2-2. 한국어 토크나이저 Okt사용


In [11]:
!pip install KoNLPy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.1 MB/s eta 0:00:00


In [12]:
from konlpy.tag import Okt
okt = Okt()

In [13]:
tokenized_sentence_list = []

for sentence in sentence_list:
    tokenized_sentence = okt.nouns(sentence)
    tokenized_sentence_list.append(tokenized_sentence)
# tokenized_sentence_list

In [14]:
len(tokenized_sentence_list)

print(tokenized_sentence_list[0])


['김치', '볶음', '거', '밥', '카레', '해먹']


### 2-2. 불용어, 특수문자 제거
* okt에서 명사만 뽑았기 때문에 특수문자 제거는 생략

In [16]:
added_sw = ['거', '점', '걸', '더', '게', '것', '데', '참', '뭘', '쭉']
stopwords_list = set(open('/content/drive/MyDrive/KDT/자연어 처리/data/stopword.txt').read().split('\n')).union(added_sw)
len(stopwords_list)
pre_sentence_list = []
for sentence in tokenized_sentence_list:
    word_list = []
    for word in sentence:
        if word not in stopwords_list:
            word_list.append(word)

    pre_sentence_list.append(word_list)

len(pre_sentence_list)

75384

In [17]:
# 제거 후 확인
print(pre_sentence_list[:5])

[['김치', '볶음', '밥', '카레', '해먹'], ['전과', '전적', '과로', '전과', '해도', '학기', '학점', '재', '편입'], ['예시', '노트북', '용도', '컴터', '용', '시퓨'], ['의자', '빈'], ['수석', '장미', '빛', '미래']]


In [18]:
sentence_list = []
for ps in pre_sentence_list:
    if len(ps) != 0:
        sentence_list.append(ps)

print(len(sentence_list))

75363


In [19]:
print(sentence_list[:5])

[['김치', '볶음', '밥', '카레', '해먹'], ['전과', '전적', '과로', '전과', '해도', '학기', '학점', '재', '편입'], ['예시', '노트북', '용도', '컴터', '용', '시퓨'], ['의자', '빈'], ['수석', '장미', '빛', '미래']]


### 2-3. Tokenizer로 단어에 라벨링

In [20]:
# pre_sentence_list를 가지고 라벨링
# 지금 리스트니까 str 문장으로 변환해야할까?

from tensorflow.keras.preprocessing.text import Tokenizer

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence_list)

In [22]:
word2idx = tokenizer.word_index
# print(word2idx)
print(len(word2idx))

33989


In [23]:
idx2word = {value : key for key, value in word2idx.items()}

vocab_size = len(idx2word)
print(vocab_size)

33989


In [24]:
encoded = tokenizer.texts_to_sequences(sentence_list)
print(encoded[:2])

[[48, 129, 19, 484, 659], [3845, 4161, 10550, 3845, 120, 3305, 8000, 35, 2675]]


In [25]:
len(encoded)

75363

### 2-4. skipgrams로 임베딩 준비

In [26]:
from tensorflow.keras.preprocessing.sequence import skipgrams

In [27]:
# edcoded 총 개수가 33997.
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=5) for sample in encoded[:500]]
print(f'{len(skip_grams)}')

500


In [28]:
for i, sg in enumerate(skip_grams):
    if len(sg[0]) <= 2:
        print(f'{i}: {sg}')

# 빈 값이 있는 것 같아서 확인함

223: ([], [])
428: ([], [])


### 2-5. 학습을 위한 모델 만들기

In [29]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dot
from tensorflow.keras.utils import plot_model

In [30]:
embedding_dim = 100

w_inputs = Input(shape=(1,), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

c_inputs = Input(shape=(1,), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_dim)(c_inputs)


In [31]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 100)               3398900   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 100)               3398900   ['input_2[0][0]']             
                                                                                              

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam')

### 2-6. 학습

In [33]:
import numpy as np

In [34]:
for epoch in range(100):
    loss = 0
    for _, elem in enumerate(skip_grams):
        if elem[0]:
            first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
            second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
            # first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
            # second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
            labels = np.array(elem[1], dtype='int32')
            X = [first_elem, second_elem]
            Y = labels
            loss += model.train_on_batch(X, Y)
    print('Epoch: ', epoch+1, 'Loss: ', loss)


Epoch:  1 Loss:  345.18286603689194
Epoch:  2 Loss:  334.79369938373566
Epoch:  3 Loss:  308.41723173856735
Epoch:  4 Loss:  260.68071299791336
Epoch:  5 Loss:  202.14269189536572
Epoch:  6 Loss:  146.9186950325966
Epoch:  7 Loss:  102.48286475986242
Epoch:  8 Loss:  70.11125607416034
Epoch:  9 Loss:  47.86330188438296
Epoch:  10 Loss:  32.95165177807212
Epoch:  11 Loss:  22.979411309584975
Epoch:  12 Loss:  16.24430738016963
Epoch:  13 Loss:  11.629797230940312
Epoch:  14 Loss:  8.421299951383844
Epoch:  15 Loss:  6.160351684316993
Epoch:  16 Loss:  4.548642620909959
Epoch:  17 Loss:  3.3899174239486456
Epoch:  18 Loss:  2.5486590452492237
Epoch:  19 Loss:  1.9313176575815305
Epoch:  20 Loss:  1.4777417610166594
Epoch:  21 Loss:  1.1425434833799955
Epoch:  22 Loss:  0.8943536909064278
Epoch:  23 Loss:  0.7094871142471675
Epoch:  24 Loss:  0.5698129861848429
Epoch:  25 Loss:  0.4672195416933391
Epoch:  26 Loss:  0.38639783910184633
Epoch:  27 Loss:  0.3262074086815119
Epoch:  28 Loss: 

In [39]:
import gensim


In [40]:
f = open('vectors.txt', 'w')
f.write('{} {}\n'.format(vocab_size, embedding_dim))
vectors = model.get_weights()[0]

print(vectors)

for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i-1, :])))))

[[-3.6600698e-02  3.1070184e-02 -3.6861908e-02 ... -3.1304561e-02
  -4.1251697e-02 -1.3444938e-02]
 [ 3.4184051e-01 -1.0438968e+00  6.5281022e-01 ...  6.8400669e-01
   1.3541592e+00 -1.9816297e-01]
 [ 9.2822200e-01  9.0662009e-01  1.0685352e+00 ...  9.9791902e-01
   4.6768194e-01  5.9400713e-01]
 ...
 [ 7.1386695e-03  2.8524626e-02 -4.7016729e-02 ...  1.0430168e-02
   2.8662633e-02  3.2484028e-02]
 [-3.0914461e-02 -3.4251727e-02 -6.8480894e-04 ...  3.1310964e-02
   4.7400955e-02 -2.4775757e-02]
 [ 4.8202228e-02 -4.3179035e-02  1.0251023e-02 ...  9.2054494e-03
   2.6368983e-03 -1.8279351e-02]]


In [41]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [42]:
w2v.most_similar(positive=['악어'])

[('자신감', 0.3837690055370331),
 ('시사회', 0.3658345639705658),
 ('반경', 0.3591010272502899),
 ('인문학', 0.35634472966194153),
 ('판옥선', 0.3537239730358124),
 ('갈림', 0.3450950086116791),
 ('프렌차이즈', 0.3426169753074646),
 ('교회', 0.3408311605453491),
 ('연기', 0.33708080649375916),
 ('무궁무진', 0.33647608757019043)]

In [43]:
w2v.most_similar(positive=['눈물'])

[('형님', 0.3972524404525757),
 ('광장시장', 0.38109156489372253),
 ('녹화', 0.34661513566970825),
 ('로간', 0.34656664729118347),
 ('배색', 0.3448937237262726),
 ('앱', 0.33743584156036377),
 ('빠오님', 0.3320561647415161),
 ('늘어트렸', 0.3315449357032776),
 ('카페라떼', 0.3310122787952423),
 ('이따', 0.32716938853263855)]